In [13]:
import matplotlib.pyplot as plt
import pickle

In [14]:
# Use Snakemake first to generate the pickle files which will run the benchmarks.

gpu = pickle.load(open('data/random/time_t=64_L=300_m=jax_gpu.pickle', 'rb'))
rust = pickle.load(open('data/random/time_t=64_L=300_m=rust.pickle', 'rb'))
pytorch = pickle.load(open('data/random/time_t=64_L=300_m=pytorch.pickle', 'rb'))
pytorch_gpu = pickle.load(open('data/random/time_t=64_L=300_m=pytorch_gpu.pickle', 'rb'))

In [3]:
data = gpu | rust | pytorch | pytorch_gpu

In [5]:

#Bar plot of rust vs pytorch
plt.switch_backend('Agg')  # Use non-interactive backend for saving
plt.rcParams.update({'font.size': 16})
precision = "f64"
tree_sizes = [2**i for i in range(4, 15,2)]

In [ ]:

plt.figure(figsize=(10, 5))
plt.ylabel('Gradient updates per minute')
plt.xlabel('Tree size')
plt.yscale('log')
plt.tick_params(axis='both', which='major')


plt.bar([i + 0.2 for i in range(len(tree_sizes))], [data[('rust', precision, 300, i)][0] / 20 for i in tree_sizes], width=0.2, label='PhyloGrad', color='orange', alpha=0.5)
plt.bar([i + 0.4 for i in range(len(tree_sizes))], [data[('pytorch', precision, 300, i)][0] / 20 for i in tree_sizes], width=0.2, label='Pytorch', color='green', alpha=0.5)

plt.xticks([i + 0.3 for i in range(len(tree_sizes))], tree_sizes)
plt.legend()
plt.savefig("cpu_abs.pdf", bbox_inches='tight')

In [43]:
# Memory
plt.figure(figsize=(10, 5))
plt.ylabel('Memory usage (GiB)')
plt.xlabel('Tree size')
plt.tick_params(axis='both', which='major')


plt.bar([i + 0.2 for i in range(len(tree_sizes))], [data[('rust', precision, 300, i)][1] / 1024 ** 2 for i in tree_sizes], width=0.2, label='PhyloGrad', color='orange', alpha=0.5)
plt.bar([i + 0.4 for i in range(len(tree_sizes))], [data[('pytorch', precision, 300, i)][1] / 1024 ** 2 for i in tree_sizes], width=0.2, label='Pytorch', color='green', alpha=0.5)

plt.xticks([i + 0.3 for i in range(len(tree_sizes))], tree_sizes)
plt.legend()
plt.savefig("mem_cpu_abs.pdf", bbox_inches='tight')

In [7]:
# Plot the speedup
plt.figure(figsize=(10, 5))
plt.ylabel('PhyloGrad / Pytorch')
plt.xlabel('Tree size')
plt.bar([i for i in range(len(tree_sizes))], [data[('rust', precision, 300, i)][0] / data[('pytorch', precision, 300, i)][0] for i in tree_sizes], width=0.2, color='purple', alpha=0.5)
plt.xticks([i for i in range(len(tree_sizes))], tree_sizes)
plt.savefig("cpu_ratio.pdf", bbox_inches='tight')

In [9]:
# Plot the speedup of GPU
plt.figure(figsize=(10, 5))
plt.ylabel('Gradient updates per minute')
plt.yscale('log')
plt.xlabel('Tree size')
plt.bar([i + 0.2 for i in range(len(tree_sizes))], [data[('jax_gpu', precision, 300, i)][0] / 20 for i in tree_sizes], width=0.2, label='PhyloGrad GPU', color='orange', alpha=0.5)
plt.bar([i + 0.4 for i in range(len(tree_sizes))], [data[('pytorch_gpu', precision, 300, i)][0] / 20 for i in tree_sizes], width=0.2, label='Pytorch GPU', color='green', alpha=0.5)
plt.xticks([i + 0.3 for i in range(len(tree_sizes))], tree_sizes)
plt.legend()
plt.savefig("gpu_abs.pdf", bbox_inches='tight')

In [11]:
# Ratio of GPU
plt.figure(figsize=(10, 5))
plt.ylabel('PhyloGrad GPU / Pytorch GPU')
plt.xlabel('Tree size')
plt.bar([i for i in range(len(tree_sizes) - 1)], [data[('jax_gpu', precision, 300, i)][0] / data[('pytorch_gpu', precision, 300, i)][0] for i in tree_sizes[:-1]], width=0.2, label='PhyloGrad GPU', color='purple', alpha=0.5)
plt.xticks([i for i in range(len(tree_sizes) - 1)], tree_sizes[:-1])
plt.savefig("gpu_ratio.pdf", bbox_inches='tight')